In [ ]:
!pip install torch torchvision --quiet
!pip install transformers  --quiet
!pip install pandas  --quiet
!pip install numpy  --quiet
!pip install sentencepiece  --quiet
!pip install sentence-splitter  --quiet
!pip install shap --quiet





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imports**

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW,AutoModelForQuestionAnswering, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DebertaTokenizer, DebertaModel, BartTokenizer
import math

# **Model loading**

In [ ]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model.to(device) # Send the model to the GPU if we have one


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
import pandas as pd

df=pd.read_csv('./new_data_mutagenicity.csv')

# Apply the function to the column
df['label'] = df['label'].apply(lambda x: x.startswith("['True"))
#df['label'] = df['label'].apply(lambda x: x[0] == "['False - AMES non Mutagenic']")

new_column_names = {'text': 'Abstract', 'label': 'AMES'}
df.rename(columns=new_column_names, inplace=True)


# **Prediction**

In [ ]:
def predict(passage,question):
  sequence = tokenizer.encode_plus(passage,question, return_tensors="pt")['input_ids'].to(device)

  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")

  if (proba_yes >= proba_no):
    return True
  else:
    return False






In [ ]:
filter=list(df.index)

In [ ]:
preds=[]
index=[]
for i in filter:
    try:
      passage=df.Abstract[i]
      question =df.questions[i]
      answer=predict(passage,question)
      preds.append(answer)
      index.append(i)
    except Exception as e:
      print(e)
      continue



Question: Is AZS mutagenic for TA98 or TA100 strain?, Yes: 0.43, No: 0.57
Question: Is Melamine mutagenic for TA98 or TA100 or TA1535 or TA102 strain?, Yes: 0.46, No: 0.54
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.46, No: 0.54
Question: Is 1-NP mutagenic for TA98 or TA100 or TA1535 strain?, Yes: 0.47, No: 0.53
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.48, No: 0.52
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.48, No: 0.52
Question: Is 1-NP mutagenic for TA98 or TA100 strain?, Yes: 0.47, No: 0.53
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.48, No: 0.52
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.45, No: 0.55
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.44, No: 0.56
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.44, No: 0.56
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.42, No: 0.58
Question: Is 1-NP mutagenic for TA98 or TA100 strain?, Yes: 0.47, No: 0.53
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.44, No: 0.56

In [ ]:
len(index)


1475

In [ ]:
df=df[df.index.isin(index)]
true_results=df['AMES'].tolist()

In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds,output_dict=True))

{'False': {'precision': 0.4448476052249637, 'recall': 0.9287878787878788, 'f1-score': 0.6015701668302257, 'support': 660}, 'True': {'precision': 0.5154639175257731, 'recall': 0.06134969325153374, 'f1-score': 0.10964912280701755, 'support': 815}, 'accuracy': 0.44949152542372883, 'macro avg': {'precision': 0.4801557613753684, 'recall': 0.4950687860197063, 'f1-score': 0.3556096448186216, 'support': 1475}, 'weighted avg': {'precision': 0.4838661099877839, 'recall': 0.44949152542372883, 'f1-score': 0.3297629458953683, 'support': 1475}}


In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds))

              precision    recall  f1-score   support

       False       0.44      0.93      0.60       660
        True       0.52      0.06      0.11       815

    accuracy                           0.45      1475
   macro avg       0.48      0.50      0.36      1475
weighted avg       0.48      0.45      0.33      1475

